<a href="https://colab.research.google.com/github/cameron-crimins-pahl/abm/blob/master/hips_dont_lie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mujoco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 14.7 MB/s eta 0:00:00


In [2]:
!pip install mediapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00


In [3]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 25.5 MB/s eta 0:00:00


In [4]:
#!pip install 'shimmy>=0.2.1'
!pip install gymnasium==0.29
!pip install shimmy==1.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.1
    Uninstalling gymnasium-0.29.1:
      Successfully uninstalled gymnasium-0.29.1


In [5]:
import os
import mujoco
import mujoco.viewer
import numpy as np
import pandas as pd
import mediapy as media
import argparse
from stable_baselines3 import SAC, TD3, A2C
from stable_baselines3.common.env_util import make_vec_env
# from hdl_v8_env import CustomMuJoCoEnv, EpisodeTrackerCallback
from stable_baselines3.common.vec_env import SubprocVecEnv
import random
import time
from gym import spaces
from gym import Env
from stable_baselines3.common.callbacks import BaseCallback
import xml.etree.ElementTree as ET
import shutil

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [6]:
!pwd

/content


In [7]:
model_dir = "models"
log_dir = "logs"
data_dir = "data"
data_multi_env = "data/multi_env"
data_single_env = "data/single_env"
xml = "xml"
stl = "stl"

os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
os.makedirs(xml, exist_ok=True)

os.makedirs(data_multi_env, exist_ok=True)
os.makedirs(data_single_env, exist_ok=True)
os.makedirs(stl, exist_ok=True)



#add xml and stl files


In [8]:
tree = ET.parse("/content/xml/trexml_LOCAL.xml")
root = tree.getroot()

In [10]:
def get_actuators(data):

    ids = {}
    num = 0
    for motor in root.findall(".//motor"):
        motor_name = motor.get("name")
        if motor_name is not None:
            # print(motor_name)
            num = num + 1
            ids.update({motor_name: data.actuator(motor_name).id})

    if len(ids) < 1:
        for motor in root.findall(".//general"):
            motor_name = motor.get("name")
            if motor_name is not None:
                # print(motor_name)
                num = num + 1
                ids.update({motor_name: data.actuator(motor_name).id})
        # ids.append(data.joint(joint_name).id)

    # print("number")
    # print(num)
    # print(ids)
    return ids

In [12]:
def get_joint_ids_names(model):

    names = {}

    for joint in root.findall(".//joint"):

        joint_name = joint.get("name")

        if joint_name is not None:
          # print(model.joint(joint_name).id)
          # print(joint_name)
          names[model.joint(joint_name).id] = joint_name
          # print(names)
          # names.append(joint_name)
          # names.update( {model.joint(joint_name).id,joint_name} )



            # b = (joint_name, model.joint(joint_name).id)

            # """model joint pos == the x y and z coordinates
            #     model joint range == the -1 to 1 range of joint motion ; """

            # names.append(b)

    return names

In [13]:
def get_joint_names_positions(model):

    names = []

    for joint in root.findall(".//joint"):

        joint_name = joint.get("name")

        if joint_name is not None:

            b = (model.joint(joint_name).id, joint_name, model.joint(joint_name).pos)

            """model joint pos == the x y and z coordinates
                model joint range == the -1 to 1 range of joint motion ; """

            names.append(b)

    return names

In [20]:
class CustomMuJoCoEnv(Env):
    def __init__(self, xml_file, render=False):
        self.model = mujoco.MjModel.from_xml_path(xml_file)
        self.data = mujoco.MjData(self.model)
        # self.env_id = env_id

        # Define observation and action spaces
        obs_size = self.model.nq + self.model.nv  # qpos (positions) + qvel (velocities)
        act_size = self.model.nu  # number of actuators




        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.model.nq + self.model.nv,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(act_size,), dtype=np.float32)  # Set the range for actions

    def get_hips_id(self):
        hip_id = self.data.geom("HIPS").id
        # hip_height = env.data.geom_xpos[hip_id][2]
        return hip_id

    def get_hips_height(self):
        hip_id = self.data.geom("HIPS").id
        hip_height = self.data.geom_xpos[hip_id][2]
        return hip_height

    def reset(self):
        mujoco.mj_resetData(self.model, self.data)
        # observation = np.concatenate([self.data.qpos, self.data.qvel])
        return self.get_obs()

    def step(self, action):

        self.data.ctrl[:] = action
        mujoco.mj_step(self.model, self.data)
        obs = self.get_obs()
        done = self.is_done()

        reward = self.compute_reward()

        try:
            hips_height = self.get_hips_height()
        except Exception as e:
            hips_height = 2.9568

        try:
            joint_torques = self.calculate_joint_torque()

        except Exception as e:
            dkt = get_joint_ids_names(self.model)
            joint_torques = {f'joint_{dkt[i]}': 0.0 for i in range(self.model.njnt)}

        # info = {'joint_torques': joint_torques, 'hips_height': hips_height, 'env_id':self.env_id}
        info = {'joint_torques': joint_torques, 'hips_height': hips_height}

        return obs, reward, done, info

    def get_obs(self):
        # np.concatenate([self.data.qpos, self.data.qvel])
        return np.concatenate([self.data.qpos.ravel(), self.data.qvel.ravel()])

    def compute_reward(self):

        HIPS_geom_id = self.data.geom("HIPS").id
        #head_body_id = self.data.geom("skull").id
        #hip_forward_velocity = self.data.cvel[HIPS_geom_id][0]
        hip_forward_velocity = -self.data.qvel[0]


        torso_height = self.data.geom_xpos[HIPS_geom_id][2]
        reward = hip_forward_velocity

        # head_forward_velocity = self.data.xvelp[head_body_id][0]

        if hip_forward_velocity >0:
        # if head_forward_velocity > 0:
            reward += 100

        if torso_height>2:
            reward +=150

        return reward

    def is_done(self):
        # is this episode done/ if the robot has fallen
        # self.data.geom_xpos[HIPS_geom_id]
        # if self.data.qpos[2] < .1:
        HIPS_geom_id = self.data.geom("HIPS").id
        hip_height =self.data.geom_xpos[HIPS_geom_id][2]
        # print(hip_height)

        if hip_height<1.0:
            # print("hip height")
            # print(self.data.geom_xpos[1][2])
            return True

        return False

    # def render(self, mode="human"):
    #     if self.viewer:
    #         self.viewer.render()
    def calculate_joint_torque(self):
        # Calculate joint torques and return them as a dictionary

        joint_torques_s = {"joint_torques":{}}

        for i in range(self.model.njnt-1):

            """ i is the id, joint name is the name, and torque is torque by i
            so i need a dict with name:torque and that's it I think"""

            joint_name = mujoco.mj_id2name(self.model, mujoco.mjtObj.mjOBJ_JOINT, i)

            if joint_name:

                commanded_torque = self.data.ctrl[i]  # Commanded torque from actuators

                """get the torques of each joint id"""
                actual_torque = self.data.qfrc_actuator[i+6]
                # actual_force = self.data.actuator_force[i]  # Actual force applied by the actuator
                joint_torques_s[joint_name] = {
                                                "commanded_torque": commanded_torque,
                                                "actual_force": actual_torque}
                # joint_torques_s.update({joint_name:actual_torque})

        """ self.data.actuator_force is length 54 because it doesn't have roots.
        the only thing with 60 is the qfrc_actuator
            """
        # print("\njoint_torques")
        # print(type(joint_torques_s))
        # print(joint_torques_s)
        return joint_torques_s


    def render(self, mode='rgb_array'):
        if mode == "rgb_array":
            if not hasattr(self, "renderer"):
                self.renderer = mujoco.Renderer(self.model,height=720, width=1080)

            self.renderer.update_scene(self.data)
            return self.renderer.render()

        pass

    def close(self):
        if self.viewer:
            self.viewer.close()
        pass

In [22]:
def log_joint_torques_to_csv(episode_num, joint_torques ,timestep, reward, hip_height):
# def log_joint_torques_to_csv(episode_num, joint_torques ,timestep, reward, hip_height):

    file_path = f"/content/data/multi_env/joint_torques_{episode_num}.csv"

    if os.path.exists(file_path):


        headers=False

    else:

        headers=True

    # dkt = {"episode":[episode_num], "Timestep":[timestep], "reward":[reward],"hip_height":[hip_height], "env_id":[env_id]}

    dkt = {"episode":[episode_num], "Timestep":[timestep], "reward":[reward],"hip_height":[hip_height]}

    df1= pd.json_normalize(joint_torques)

    df2=pd.DataFrame(dkt)

    df = pd.concat([df1,df2],axis=1)
    # print(df)

    df.to_csv(file_path, mode="a", header=headers)

In [23]:
def adjust_joint_control(model, data, enabled_joint_indices):

    for i in range(model.nu):  # 'nu' is the number of actuators
        if i not in enabled_joint_indices:
            data.ctrl[i] = 0  # Disable control for this joint

In [24]:
def train_single_env(xml_file_path, sb3_algo):
  env = CustomMuJoCoEnv(xml_file_path, render=False)


  # Create the model based on the algorithm
  match sb3_algo:
      case 'SAC':
          model = SAC('MlpPolicy', env, verbose=1, device="cuda", tensorboard_log=log_dir)
      case 'TD3':
          model = TD3('MlpPolicy', env, verbose=1, device="cuda", tensorboard_log=log_dir)
      case 'A2C':
          model = A2C('MlpPolicy', env, verbose=1, device="cuda", tensorboard_log=log_dir)
      case _:
          print('Algorithm not found')
          return

  # Number of timesteps per iteration
  #TIMESTEPS = 25000
  TIMESTEPS_PER_EPISODE = []
  initial_timesteps = 5000
  final_timesteps = 20000
  stagger_intervals = 1500

  step_increase = (final_timesteps - initial_timesteps) / stagger_intervals

  # Train for multiple episodes
  num_episodes = 5000
  iters = 0

  #step_increase = (final_timesteps - initial_timesteps) / stagger_intervals

  for episode_count in range(num_episodes):

      """  episode staggered intervals """
      if episode_count < stagger_intervals:
        # if ep 3 is less than 1500:
        # current timestep = 5000 + 3 * 19996

        current_timesteps = int(initial_timesteps + episode_count * step_increase)

      else:

        current_timesteps = final_timesteps

      TIMESTEPS_PER_EPISODE.append(current_timesteps)

      current_timesteps = TIMESTEPS_PER_EPISODE[episode_count]

      # else:
        # enabled_joints = ["femurL_hip_joint1","femurL_hip_joint2","tibiaL_knee_joint"

      print(f"Episode {episode_count + 1}/{num_episodes}")
      obs = env.reset()

      done = False
      episode_reward = 0
      episode_timesteps = 0

      episode_data = []

      while not done:

          action, _ = model.predict(obs)
          obs, reward, done, info = env.step(action)

          joint_torques = info["joint_torques"]

          hip_id = env.data.geom("HIPS").id
          hip_height = env.data.geom_xpos[hip_id][2]

          survival_reward = 100
          reward += survival_reward
          reward += episode_timesteps

          #episode_reward += reward

          # log_joint_torques_to_csv(episode_num=episode_count, joint_torques=joint_torques, timestep=episode_timesteps , reward=reward, hip_height=hip_height)
          log_joint_torques_to_csv(episode_num=episode_count, joint_torques=joint_torques, timestep=episode_timesteps , reward=reward, hip_height=hip_height)

          episode_reward += reward[0]  # Accumulate rewards
          episode_timesteps += 1

      print(f"Episode {episode_count + 1}: Reward = {episode_reward}, Timesteps = {episode_timesteps}")
      iters += 1

      model.learn(total_timesteps=current_timesteps, reset_num_timesteps=False)

      obs = env.reset()

      model.save(f"{model_dir}/{episode_reward}_{sb3_algo}_{current_timesteps * (episode_count + 1)}.zip")

  print("Training complete.")

In [18]:
sb3_algo = 'SAC'

In [19]:
xml_file_path = "/content/xml/trexml_LOCAL.xml"

In [ ]:
#/usr/local/lib/python3.10/dist-packages/shimmy/openai_gym_compatibility.py in step(self, action)
#add timestep to self.gym_env.step(action) ?
train_single_env(xml_file_path, sb3_algo)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Episode 1/5000


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Episode 1: Reward = 57981.37061785602, Timesteps = 251
Logging to logs/SAC_0
less than .5
-1.9165356540716851
less than .5
-1.9013061067687753
less than .5
-1.8900057423235255
less than .5
-1.883309600668514
less than .5
-1.8657077154451571
less than .5
-1.8701447066085983
less than .5
-1.873423045398742
less than .5
-1.856734504363867
less than .5
-1.8504473432287587
less than .5
-1.6564107050216088
less than .5
-1.5456507222502651
less than .5
-1.502227244398683
less than .5
-1.4825504106696616
less than .5
-1.472452479027379
less than .5
-0.5429874728370503
less than .5
-0.5139688377474418
less than .5
-0.6379505882348459
less than .5
-0.7398906382164183
less than .5
-0.8246409845298686
less than .5
-0.9055913090885805
less than .5
-0.972995535956263
less than .5
-1.0441381633011582
less than .5
-1.1049415487858034
less than .5
-1.1478180822921882
less than .5
-1.1969097182210118
less than .5
-1.218052424310004
less than .5
-1.2545437279556628
less than .5
-1.2676966320121912
less t

KeyboardInterrupt: 